In [1]:
import sys

sys.path.append("..")

In [2]:
import pandas as pd
print(pd.__version__)

1.1.5


In [3]:

!git clone https://github.com/AI4Finance-LLC/FinRL-Library.git

Cloning into 'FinRL-Library'...
remote: Enumerating objects: 2824, done.
remote: Total 2824 (delta 0), reused 0 (delta 0), pack-reused 2824
Receiving objects: 100% (2824/2824), 30.68 MiB | 34.71 MiB/s, done.
Resolving deltas: 100% (1697/1697), done.


In [4]:
%cd FinRL-Library/
!pip install -r requirements.txt
!pip install nest-asyncio

/content/FinRL-Library
     |████████████████████████████████| 153kB 26.9MB/s 
     |████████████████████████████████| 5.5MB 45.1MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=c31f0472f9f449a1346c9ebb59578d376636eb2a8388996f96628f920738819e
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


/content/FinRL-Library

In [5]:
%cd FinRL-Library/

[Errno 2] No such file or directory: 'FinRL-Library/'
/content/FinRL-Library


In [6]:
import nest_asyncio
nest_asyncio.apply()

In [7]:
%pip install pyfolio 

     |████████████████████████████████| 92kB 8.9MB/s 
     |████████████████████████████████| 61kB 9.9MB/s 
  Created wheel for pyfolio: filename=pyfolio-0.9.2-cp37-none-any.whl size=88667 sha256=90956bc8d294e7d394106ad76664ceb9a163d0ba108b46a358997dbf6b311c81
  Stored in directory: /root/.cache/pip/wheels/0d/a8/7d/ac33fe6bf6f3db370291c9de303460d00b8f97cc40a1d80f72
  Created wheel for empyrical: filename=empyrical-0.5.5-cp37-none-any.whl size=39764 sha256=fc7008c8d7811932a279f8b350ab456417f50ae43d8134b7cf227edc75f7738d
  Stored in directory: /root/.cache/pip/wheels/ea/b2/c8/6769d8444d2f2e608fae2641833110668d0ffd1abeb2e9f3fc
Successfully built pyfolio empyrical


In [8]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.config import config
from finrl.marketdata.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.env_stocktrading import StockTradingEnv
from finrl.model.models import DRLAgent
from finrl.trade.backtest import backtest_stats, backtest_plot, get_daily_return, get_baseline

from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

/usr/local/lib/python3.7/dist-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; mutltipliers will not be applied' +


In [9]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

In [10]:
!pip install -U ipython

     |████████████████████████████████| 788kB 17.9MB/s 
     |████████████████████████████████| 368kB 50.1MB/s 
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 3.0.16 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.21.0 which is incompatible.
  Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0


In [11]:
!pip install finrl

In [12]:
#from finrl.config.configuration import Configuration
#from finrl.config.directory_operations import create_userdata_dir
#from finrl.commands import start_download_cryptodata, start_download_stockdata

In [12]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.config import config
from finrl.marketdata.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
#from finrl.env.env_stocktrading_v2 import StockTradingEnvV2
from finrl.model.models import DRLAgent
#from finrl.trade.backtest import BackTestStats, BaselineStats, BackTestPlot

from pprint import pprint

In [13]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

In [14]:
# from config.py start_date is a string
config.START_DATE

'2000-01-01'

In [15]:
# from config.py end_date is a string
config.END_DATE

'2021-01-01'

In [ ]:
print(config.DOW_30_TICKER)

In [17]:
df = YahooDownloader(start_date = '2009-01-01',
                     end_date = '2021-01-01',
                     ticker_list = config.DOW_30_TICKER).fetch_data()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [18]:
df.shape

(90630, 8)

In [19]:
df.sort_values(['date','tic'],ignore_index=True).head()


,date,open,high,low,close,volume,tic,day
0,2009-01-02,3.067143,3.251429,3.041429,2.791740,746015200,AAPL,4
1,2009-01-02,18.570000,19.520000,18.400000,15.745411,10955700,AXP,4
2,2009-01-02,42.799999,45.560001,42.779999,33.941101,7010200,BA,4
3,2009-01-02,44.910000,46.980000,44.709999,32.978760,7117200,CAT,4
4,2009-01-02,16.410000,17.000000,16.250000,12.683227,40980600,CSCO,4


In [20]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.TECHNICAL_INDICATORS_LIST,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)


Successfully added technical indicators
Successfully added turbulence index


In [22]:
processed.sort_values(['date','tic'],ignore_index=True).head(10)

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2009-01-02,3.067143,3.251429,3.041429,2.791740,746015200,AAPL,4,0.000000,3.017278,2.684025,100.000000,66.666667,100.000000,2.791740,2.791740,0.0
1,2009-01-02,18.570000,19.520000,18.400000,15.745411,10955700,AXP,4,0.002643,3.017278,2.684025,100.000000,66.666667,100.000000,2.850651,2.850651,0.0
2,2009-01-02,42.799999,45.560001,42.779999,33.941101,7010200,BA,4,0.001880,2.972787,2.735796,70.355711,46.771878,100.000000,2.854292,2.854292,0.0
3,2009-01-02,44.910000,46.980000,44.709999,32.978760,7117200,CAT,4,-0.000746,2.951725,2.729582,50.429389,-29.777993,43.607834,2.840654,2.840654,0.0
4,2009-01-02,16.410000,17.000000,16.250000,12.683227,40980600,CSCO,4,-0.000088,2.939568,2.746169,60.227126,-9.019317,48.357918,2.842869,2.842869,0.0
5,2009-01-02,74.230003,77.300003,73.580002,47.373642,13695900,CVX,4,-0.002480,2.931445,2.735506,47.932736,-27.237701,49.952380,2.833476,2.833476,0.0
6,2009-01-02,21.605234,22.060680,20.993229,15.336650,13251037,DD,4,-0.006646,2.938422,2.698235,40.237370,-101.614424,9.320766,2.818329,2.818329,0.0
7,2009-01-02,22.760000,24.030001,22.500000,20.597496,9796600,DIS,4,-0.010724,2.943226,2.663404,37.181826,-113.576183,3.614243,2.803315,2.803315,0.0
8,2009-01-02,84.019997,87.620003,82.190002,72.559456,14088500,GS,4,-0.017016,2.960305,2.606701,31.067739,-136.004971,17.746098,2.783503,2.783503,0.0
9,2009-01-02,23.070000,24.190001,22.959999,17.909452,14902500,HD,4,-0.024322,2.978173,2.545136,27.267345,-166.471239,43.235022,2.761654,2.761654,0.0


In [23]:
train = data_split(processed, '2009-01-01','2019-01-01')
trade = data_split(processed, '2019-01-01','2021-01-01')
print(len(train))
print(len(trade))

75480
15150


In [24]:
import time
milliseconds = int(round(time.time() * 1000))
print(milliseconds)

1614831458901


In [25]:
import numpy as np
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces
import matplotlib
from copy import deepcopy

matplotlib.use("Agg")
import matplotlib.pyplot as plt
import pickle
from stable_baselines3.common.vec_env import DummyVecEnv
import numpy as np
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces
import matplotlib
import random

matplotlib.use("Agg")
import matplotlib.pyplot as plt
import pickle
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common import logger
import time

class StockTradingEnvV2(gym.Env):
    
    """
    A stock trading environment for OpenAI gym
    Parameters:
    state space: {start_cash, <owned_shares>, for s in stocks{<stock.values>}, }
        df (pandas.DataFrame): Dataframe containing data
        transaction_cost (float): cost for buying or selling shares
        hmax (int): max number of share purchases allowed per asset
        turbulence_threshold (float): Maximum turbulence allowed in market for purchases to occur. If exceeded, positions are liquidated
        print_verbosity(int): When iterating (step), how often to print stats about state of env
        reward_scaling (float): Scaling value to multiply reward by at each step. 
        initial_amount: (int, float): Amount of cash initially available
        daily_information_columns (list(str)): Columns to use when building state space from the dataframe. 
        out_of_cash_penalty (int, float): Penalty to apply if the algorithm runs out of cash
    
    action space: <share_dollar_purchases>
    
    TODO:
        property for date index - starting point
        

    tests:
        after reset, static strategy should result in same metrics

        buy zero should result in no costs, no assets purchased
        given no change in prices, no change in asset values
    """
    metadata = {"render.modes": ["human"]}

    def __init__(
        self,
        df,
        transaction_cost_pct=3e-3,
        date_col_name="date",
        hmax=10,
        turbulence_threshold=None,
        print_verbosity=10,
        reward_scaling=1e-4,
        initial_amount=1e6,
        daily_information_cols=["open", "close", "high", "low", "volume"],
        out_of_cash_penalty=None,
        cache_indicator_data = True,
        daily_reward = None,
        cash_penalty_proportion = 0.1, 
        random_start = True
    ):
        self.df = df
        self.stock_col = "tic"
        self.assets = df[self.stock_col].unique()
        self.dates = df[date_col_name].sort_values().unique()
        self.random_start = random_start

        self.df = self.df.set_index(date_col_name)
        self.hmax = hmax
        self.initial_amount = initial_amount
        if out_of_cash_penalty is None:
            out_of_cash_penalty=-initial_amount*0.5
        self.out_of_cash_penalty = out_of_cash_penalty
        self.print_verbosity = print_verbosity
        self.transaction_cost_pct = transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.daily_information_cols = daily_information_cols
        self.close_index = self.daily_information_cols.index("close")
        self.state_space = (
            1 + len(self.assets) + len(self.assets) * len(self.daily_information_cols)
        )
        self.action_space = spaces.Box(low=-1, high=1, shape=(len(self.assets),))
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(self.state_space,)
        )
        self.episode = -1  # initialize so we can call reset
        self.episode_history = []
        self.printed_header = False
        self.daily_reward = daily_reward
        self.cache_indicator_data = cache_indicator_data
        self.cached_data = None
        self.cash_penalty_proportion = cash_penalty_proportion
        if self.cache_indicator_data:
            print("caching data")
            self.cached_data = [self.get_date_vector(i) for i, _ in enumerate(self.dates)]
            print("data cached!")
        

    def seed(self, seed=None):
        if seed is None:
            seed = int(round(time.time() * 1000))
        random.seed(seed)
        
    @property
    def current_step(self):
        return self.date_index-self.starting_point


    def reset(self):
        self.seed()
        self.sum_trades = 0
        if self.random_start:
            starting_point = random.choice(range(int(len(self.dates)*0.5)))
            self.starting_point = starting_point
        else:
            self.starting_point = 0
        self.date_index = self.starting_point
        self.episode += 1
        self.actions_memory = []
        self.transaction_memory = []
        self.state_memory = []
        self.account_information = {
            "cash": [],
            "asset_value": [],
            "total_assets": [],
            'reward': []
        }
        init_state = np.array(
                [self.initial_amount]
                + [0] * len(self.assets)
                + self.get_date_vector(self.date_index)
            )
        self.state_memory.append(init_state)
        return init_state

    def get_date_vector(self, date, cols=None):
        if (cols is None) and (self.cached_data is not None):
            return self.cached_data[date]
        else:
            date = self.dates[date]
            if cols is None:
                cols = self.daily_information_cols
            trunc_df = self.df.loc[date]
            v = []
            for a in self.assets:
                subset = trunc_df[trunc_df[self.stock_col] == a]
                v += subset.loc[date, cols].tolist()
            assert len(v) == len(self.assets) * len(cols)
            return v
        
    def return_terminal(self, reason='Last Date', reward=0):

        state = self.state_memory[-1]
        self.log_step(reason = reason, terminal_reward= reward)
        reward = reward*self.reward_scaling
        # Add outputs to logger interface
        reward_pct = self.account_information['total_assets'][-1]/self.initial_amount
        logger.record("environment/total_reward_pct", (reward_pct-1)*100)
        logger.record("environment/daily_trades", self.sum_trades/(self.current_step)/len(self.assets))
        logger.record("environment/completed_steps", self.current_step)
        logger.record("environment/sum_rewards", np.sum(self.account_information['reward']))
        logger.record("environment/cash_proportion", self.account_information['cash'][-1]/self.account_information['total_assets'][-1])
        return state, reward, True, {}
    
    def log_step(self, reason, terminal_reward=None):
        if terminal_reward is None:
            terminal_reward = self.account_information['reward'][-1]
        cash_pct = self.account_information['cash'][-1]/self.account_information['total_assets'][-1]
        rec = [self.episode, self.date_index-self.starting_point, reason, f"${int(self.account_information['total_assets'][-1])}",f"${terminal_reward:0.2f}", f"{cash_pct*100:0.2f}%"]

        self.episode_history.append(rec)
        print(self.template.format(*rec))

    def step(self, actions):
        #let's just log what we're doing in terms of max actions at each step. 
        self.sum_trades += np.sum(np.abs(actions))
        
        #print header only first time
        if self.printed_header is False:
            self.template = "{0:4}|{1:4}|{2:15}|{3:10}|{4:10}|{5:10}" # column widths: 8, 10, 15, 7, 10
            print(self.template.format("EPISODE", "STEPS", "TERMINAL_REASON", "TOT_ASSETS", "TERMINAL_REWARD_unsc", "CASH_PCT"))
            self.printed_header = True

        # define terminal function in scope so we can do something about the cycle being over


        # print if it's time.
        if (self.current_step+ 1) % self.print_verbosity == 0:
            self.log_step(reason = 'update')

        #if we're at the end
        if self.date_index == len(self.dates) - 1:
            #if we hit the end, set reward to total gains (or losses)
            terminal_reward = self.account_information['total_assets'][-1]-self.initial_amount
            return self.return_terminal(reward = terminal_reward)
        else:
            #compute value of cash + assets
            begin_cash = self.state_memory[-1][0]
            holdings = self.state_memory[-1][1 : len(self.assets) + 1]
            assert (min(holdings)>=0)
            closings = np.array(self.get_date_vector(self.date_index, cols=["close"]))
            asset_value = np.dot(holdings, closings)

            # reward is (cash + assets) - (cash_last_step + assets_last_step)
            if self.current_step==0:
                reward = 0
            else:
                #stepwise reward
                reward = (
                    begin_cash + asset_value - self.account_information["total_assets"][-1]
                )
                #
                if self.daily_reward is not None:
                    reward+=(self.daily_reward*self.current_step)
                if self.cash_penalty_proportion is not None:
                    cash_penalty = min(0, begin_cash-(self.cash_penalty_proportion*self.initial_amount))
                    reward += cash_penalty

            # log the values of cash, assets, and total assets
            self.account_information["cash"].append(begin_cash)
            self.account_information["asset_value"].append(asset_value)
            self.account_information["total_assets"].append(begin_cash + asset_value)
            self.account_information['reward'].append(reward)

            # multiply action values by our scalar multiplier and save
            actions = actions * self.hmax
            self.actions_memory.append(actions)
            
            
            #scale cash purchases to asset # changes
            actions = actions/closings 
            self.transaction_memory.append(actions)
            
            
            # clip actions so we can't sell more assets than we hold
            actions = np.maximum(actions, -np.array(holdings))


            # compute our proceeds from sales, and add to cash
            sells = -np.clip(actions, -np.inf, 0)
            proceeds = np.dot(sells, closings)
            costs = proceeds * self.transaction_cost_pct
            coh = begin_cash + proceeds

            # compute the cost of our buys
            buys = np.clip(actions, 0, np.inf)
            spend = np.dot(buys, closings)
            costs += spend * self.transaction_cost_pct

            # if we run out of cash, end the cycle and penalize
            if (spend + costs) > coh:
                return self.return_terminal(reason = 'CASH SHORTAGE',
                    reward=self.out_of_cash_penalty,
                )

            # verify we didn't do anything impossible here
            assert (spend + costs) <= coh

            # update our holdings
            coh = coh - spend - costs
            holdings_updated = holdings + actions
            self.date_index += 1
            state = (
                [coh] + list(holdings_updated) + self.get_date_vector(self.date_index)
            )
            self.state_memory.append(state)
            reward = reward * self.reward_scaling
            return state, reward, False, {}

    def get_sb_env(self):
        def get_self():
            return deepcopy(self)
        e = DummyVecEnv([get_self])
        obs = e.reset()
        return e, obs
    
    def get_multiproc_env(self, n = 10):
        def get_self():
            return deepcopy(self)
        e = SubprocVecEnv([get_self for _ in range(n)], start_method = 'fork')
        obs = e.reset()
        return e, obs

    def save_asset_memory(self):
        if self.current_step==0:
            return None
        else:
            self.account_information["date"] = self.dates[- len(self.account_information['cash']):]
            return pd.DataFrame(self.account_information)

    def save_action_memory(self):
        if self.current_step==0:
            return None
        else:
            return pd.DataFrame(
                {"date": self.dates[-len(self.account_information['cash']): ], 
                 "actions": self.actions_memory, 
                 "transactions": self.transaction_memory}
            )

In [26]:
print(StockTradingEnvV2.__doc__)


    A stock trading environment for OpenAI gym
    Parameters:
    state space: {start_cash, <owned_shares>, for s in stocks{<stock.values>}, }
        df (pandas.DataFrame): Dataframe containing data
        transaction_cost (float): cost for buying or selling shares
        hmax (int): max number of share purchases allowed per asset
        turbulence_threshold (float): Maximum turbulence allowed in market for purchases to occur. If exceeded, positions are liquidated
        print_verbosity(int): When iterating (step), how often to print stats about state of env
        reward_scaling (float): Scaling value to multiply reward by at each step. 
        initial_amount: (int, float): Amount of cash initially available
        daily_information_columns (list(str)): Columns to use when building state space from the dataframe. 
        out_of_cash_penalty (int, float): Penalty to apply if the algorithm runs out of cash
    
    action space: <share_dollar_purchases>
    
    TODO:
       

In [27]:
information_cols = ['open', 'high', 'low', 'close', 'volume', 'day', 'macd', 'rsi_30', 'cci_30', 'dx_30', 'turbulence']

env_kwargs = {
    "hmax": 5000, 
    "daily_reward": 5,
    "out_of_cash_penalty": 0, 
    "cash_penalty_proportion": 0.1,
    "daily_information_cols": information_cols, 
    "print_verbosity": 500, 
    
}
e_train_gym = StockTradingEnvV2(df = train, cache_indicator_data=True,
                              **env_kwargs)

# e_train_obs = StockTradingEnvV2(df = train, cache_indicator_data=False, **env_kwargs)


# e_trade_gym = StockTradingEnvV2(df = train,**env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()


caching data
data cached!


# single processing
env_train, _ = e_train_gym.get_sb_env()


#multiprocessing
env_train, _ = e_train_gym.get_multiproc_env(n = <n_cores>)

In [28]:
# for this example, let's do multiprocessing with n_cores-2

import multiprocessing

n_cores = multiprocessing.cpu_count() - 2
n_cores = 24
print(f"using {n_cores} cores")

# 
e_train_gym.print_verbosity = 500
env_train, _ = e_train_gym.get_multiproc_env(n = n_cores)
# env_train, _ = e_train_gym.get_sb_env()

env_train_obs, _ = e_train_gym.get_sb_env()

using 24 cores


In [29]:
agent = DRLAgent(env = env_train)

In [30]:
print(config.PPO_PARAMS)

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 64}


In [31]:
from torch.nn import Softsign, ReLU
ppo_params ={'n_steps': 256, 
             'ent_coef': 0.01, 
             'learning_rate': 0.00001, 
             'batch_size': 256, 
            'gamma': 0.99}

policy_kwargs = {
#     "activation_fn": ReLU,
    "net_arch": [1024, 1024, 1024,  1024], 
#     "squash_output": True
}

model = agent.get_model("ppo",  model_kwargs = ppo_params, policy_kwargs = policy_kwargs, verbose = 0)

{'n_steps': 256, 'ent_coef': 0.01, 'learning_rate': 1e-05, 'batch_size': 256, 'gamma': 0.99}


In [32]:
model.learn(total_timesteps = 1000000, 
            eval_env = env_train_obs, 
            eval_freq = 1000,
            log_interval = 1, 
            tb_log_name = 'cashbuffer_1_16_longrun',
            n_eval_episodes = 1,
            reset_num_timesteps = True)

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/callbacks.py:327: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.subproc_vec_env.SubprocVecEnv object at 0x7f5c2bb9f3d0> != <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x7f5c2bb9f650>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")
/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


EPISODE|STEPS|TERMINAL_REASON|TOT_ASSETS|TERMINAL_REWARD_unsc|CASH_PCT  
   1| 499|update         |$1056405  |$7250.51  |66.02%    
   1| 999|update         |$1193306  |$5044.88  |35.43%    
   1|1490|Last Date      |$1419759  |$419759.23|7.31%     
Eval num_timesteps=24000, episode_reward=638.60 +/- 0.00
Episode length: 1491.00 +/- 0.00
New best mean reward!
   3| 499|update         |$999495   |$2766.87  |90.42%    
   3| 999|update         |$1029358  |$5101.04  |72.42%    
   3|1492|Last Date      |$1039647  |$39647.31 |52.22%    
Eval num_timesteps=48000, episode_reward=564.07 +/- 0.00
Episode length: 1493.00 +/- 0.00
   5| 499|update         |$999613   |$2489.89  |99.98%    
   5| 999|update         |$999592   |$4990.00  |100.00%   
   5|1499|update         |$1000615  |$7569.76  |99.58%    
   5|1999|update         |$1002864  |$10421.63 |96.40%    
   5|2021|Last Date      |$1000860  |$860.11   |96.61%    
Eval num_timesteps=72000, episode_reward=1020.78 +/- 0.00
Episode length: 20

In [33]:
model.save("quicksave_ppo_dow_1_17.model")

In [34]:
data_turbulence = processed[(processed.date<'2019-01-01') & (processed.date>='2009-01-01')]
insample_turbulence = data_turbulence.drop_duplicates(subset=['date'])

In [35]:
insample_turbulence.turbulence.describe()

count    2516.000000
mean       33.278197
std        33.999880
min         0.000000
25%        15.233636
50%        25.166906
75%        39.289978
max       332.849950
Name: turbulence, dtype: float64

In [36]:
turbulence_threshold = np.quantile(insample_turbulence.turbulence.values,1)

In [37]:
turbulence_threshold


332.8499496926432

In [38]:
trade = data_split(processed, '2019-01-01','2021-01-01')
env_kwargs = {
    "hmax": 5000, 
    "daily_reward": 5,
    "out_of_cash_penalty": 0, 
    "cash_penalty_proportion": 0.1,
    "daily_information_cols": information_cols, 
    "print_verbosity": 50, 
    "random_start": False,
    "cache_indicator_data": False
    
}
e_trade_gym = StockTradingEnvV2(df = trade,**env_kwargs)
env_trade, obs_trade = e_trade_gym.get_sb_env()


In [39]:
print(len(e_trade_gym.dates))

505


In [40]:
df_account_value, df_actions = DRLAgent.DRL_prediction(model=model,
                        test_data = trade,
                        test_env = env_trade,
                        test_obs = obs_trade)

TypeError: ignored

In [ ]:
df_account_value.shape


In [ ]:
df_account_value.head(50)


In [ ]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = BackTestStats(account_value=df_account_value, value_col_name = 'total_assets')
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_"+now+'.csv')


In [ ]:
print("==============Compare to DJIA===========")
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
BackTestPlot(df_account_value, 
             baseline_ticker = '^DJI', 
             baseline_start = '2019-01-01',
             baseline_end = '2021-01-01', value_col_name = 'total_assets')

In [ ]:
print("==============Get Baseline Stats===========")
baesline_perf_stats=BaselineStats('^DJI',
                                  baseline_start = '2019-01-01',
                                  baseline_end = '2021-01-01')
